The code is stolen from https://github.com/uhh-lt/generative-ie/blob/master/src/masked_token_predictor_bert.py

In [1]:
# !pip install tensorflow
# !pip install git+https://github.com/IINemo/isanlp.git

In [1]:
!export CUDA_VISIBLE_DEVICES=1

from transformers import BertForMaskedLM
from transformers.tokenization_bert import BertTokenizer

model_name = 'bert-base-multilingual-uncased'
model = BertForMaskedLM.from_pretrained(model_name, cache_dir='./cache')
res = model.cuda()

tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=False, cache_dir='./cache')

In [2]:
1+1

2

In [3]:
import torch

from transformers import BertForMaskedLM, BertTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

model = BertForMaskedLM.from_pretrained(pretrained_weights)
model_dict = torch.load('../condBERTmodel.pth', map_location=device)
model.load_state_dict(model_dict, strict=False)

model.cuda();

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
old_bias = model.cls.predictions.decoder.bias.cpu().detach().numpy()

In [14]:
from tqdm.auto import tqdm, trange
from collections import defaultdict

data_folder = '../../../data/jigsaw_drg/'
toxic_train = [l.strip() for l in open(data_folder + 'train_toxic').readlines()]
nontoxic_train = [l.strip() for l in open(data_folder + 'train_normal').readlines()]

toxic_counter = defaultdict(lambda: 1)
nontoxic_counter = defaultdict(lambda: 1)

for text in tqdm(toxic_train):
    for token in tokenizer.encode(text):
        toxic_counter[token] += 1
for text in tqdm(nontoxic_train):
    for token in tokenizer.encode(text):
        nontoxic_counter[token] += 1

token_toxicities = [toxic_counter[i] / (nontoxic_counter[i] + toxic_counter[i]) for i in range(len(tokenizer.vocab))]

In [15]:
import numpy as np

token_toxicities = np.array(token_toxicities)
token_toxicities = np.maximum(0, np.log(1/(1/token_toxicities-1)))   # log odds ratio

# discourage meaningless tokens
for tok in ['.', ',', '-']:
    token_toxicities[tokenizer.encode(tok)][1] = 3

for tok in ['you']:
    token_toxicities[tokenizer.encode(tok)][1] = 0

In [16]:
model.cls.predictions.decoder.bias = torch.nn.Parameter(torch.tensor(old_bias - token_toxicities * 10).cuda())

In [81]:
from importlib import reload

In [101]:
import masked_token_predictor_bert
reload(masked_token_predictor_bert)
from masked_token_predictor_bert import MaskedTokenPredictorBert

from predict import analyze_tagged_text

predictor = MaskedTokenPredictorBert(model, tokenizer, max_len=250)

In [19]:
import os
import sys


def add_sys_path(p):
    p = os.path.abspath(p)
    print(p)
    if p not in sys.path:
        sys.path.append(p)

add_sys_path('../..')

/home/dale/dialogue-censor/style_transfer


In [20]:
import transfer_utils.masking
reload(transfer_utils.masking)
from transfer_utils.masking import print_mask, delete_mask

In [21]:
from isanlp.annotation_repr import CSentence
from isanlp.processor_sentence_splitter import ProcessorSentenceSplitter
from isanlp.en.processor_tokenizer_nltk_en import ProcessorTokenizerNltkEn
from isanlp import PipelineCommon


def create_parsers():
    return PipelineCommon([
        (ProcessorTokenizerNltkEn(), ['text'], {0 : 'tokens'}),
        (ProcessorSentenceSplitter(), ['tokens'], {0 : 'sentences'})
    ])
    return ppl

ppl = create_parsers()

In [22]:
from flair.data import Sentence
from flair.embeddings import WordEmbeddings
glove_embedding = WordEmbeddings('glove')

def embed(text):
    toks = glove_embedding.embed(Sentence(text))[0]
    return np.mean([t.embedding.cpu().numpy() for t in toks], axis=0)

def cosine(v1, v2):
    return np.dot(v1, v2) / np.sqrt(sum(v1**2) * sum(v2**2) + 1e-10)

In [23]:
import pickle
with open('../word2coef.pkl', 'rb') as f:
    word2coef = pickle.load(f)
    
def group_by_first_token(texts):
    seqs = [tokenizer.encode(x,add_special_tokens=False) for x in texts]
    grouped = defaultdict(list)
    for seq in seqs:
        grouped[seq[0]].append(seq)
    return grouped

#@title get_mask - Masks, given by dictionary
with open("../negative-words.txt", "r") as f:
    s = f.readlines()
    s = list(map(lambda x: x[:-1], s))
with open("../toxic_words.txt", "r") as f:
    ss = f.readlines()
    s += list(map(lambda x: x[:-1], ss))
neg_complex_tokens = group_by_first_token(s)

with open("../positive-words.txt", "r") as f:
    s = f.readlines()
    s = list(map(lambda x: x[:-1], s))
with open("../toxic_words.txt", "r") as f:
    ss = f.readlines()
    s += list(map(lambda x: x[:-1], ss))
pos_complex_tokens = group_by_first_token(s)


def get_mask_fast(inp : str, bad_words = neg_complex_tokens, min_bad_score=0, aggressive=True):
    sentences = [tokenizer.encode(inp, add_special_tokens=True)]
    sentences_torch = torch.tensor(sentences)
    masks = torch.zeros_like(sentences_torch)
    
    for sent_id, sent in enumerate(sentences):
        for first_tok_id, tok in enumerate(sent):
            for hypothesis in bad_words.get(tok, []):
                if sent[first_tok_id: (first_tok_id + len(hypothesis))] == hypothesis:
                    for step in range(len(hypothesis)):
                        masks[sent_id, first_tok_id + step] = 1
        if sum(masks[sent_id].numpy()) == 0 or aggressive:
            scored_words = []
            for indices, word in toks_to_words(sent):
                score = word2coef.get(word)
                if score:
                    scored_words.append([indices, word, score])
            if scored_words:
                max_score = max(s[2] for s in scored_words)
                if max_score > min_bad_score:
                    for indices, word, score in scored_words:
                        if score >= max(min_bad_score, max_score * 0.5):
                            masks[sent_id, indices] = 1

    return sentences_torch, masks

In [24]:
v = {v: k for k, v in tokenizer.vocab.items()}

def toks_to_words(token_ids):
    """ Merge subword tokens into whole words """
    indices = []
    for i, token_id in enumerate(token_ids):
        token_text = v[token_id]
        if token_text.startswith('##'):
            indices.append(i)
        else:
            if indices:
                toks = [v[token_ids[t]] for t in indices]
                word = ''.join([toks[0]] + [t[2:] for t in toks[1:]])
                yield indices, word
            indices = [i]

In [25]:
toks, masks = get_mask_fast('These filthy niggers are all beggars', neg_complex_tokens)
print(toks)
print(masks)

tensor([[  101,  2122, 18294,  9152, 13327,  2015,  2024,  2035, 11693,  6843,
          2015,   102]])
tensor([[0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0]])


In [26]:
print_mask(toks, masks, inv_voc=v)

 [CLS] these **filthy niggers** are all **beggar**s [SEP]

In [27]:
spans = []
tox = False
for idx, tok in enumerate(masks[0]):
    if not tox and tok:
        first = idx
        tox = True
    if tox and not tok:
        spans.append((first, idx))
        tox = False
if tox:
    spans.append((first, idx))
spans

[(2, 6), (8, 10)]

In [28]:
tokenizer.decode(toks[0][spans[0][0]: spans[0][1]])

'filthy niggers'

In [78]:
from predict import get_masked_tokens_from_tagged_text, preprocess_tagged_text

In [30]:
under = tokenizer.encode('_', add_special_tokens=False)[0]
toks2 = toks[0].tolist()
b, e = spans[0]
toks2.insert(e, under)
toks2.insert(e, under)
toks2.insert(b, under)
toks2.insert(b, under)

(tensor([[  101, 19424,  1044, 22571, 10085, 28884,  1010, 13350,  2024,  9202,
            102]]),
 tensor([[0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0]]))

In [23]:
get_masked_tokens_from_tagged_text('__disgusting_hypocrites__ , liberals are horrible')

([(0, 21)], 'disgusting_hypocrites , liberals are horrible')

In [69]:
tokenizer.encode('hello billy [SEP] how are you')

[101, 7592, 5006, 102, 2129, 2024, 2017, 102]

In [83]:
tagged_text = '__disgustinghypocrites__, liberals are horrible' 
get_masked_tokens_from_tagged_text(tagged_text)
preprocess_tagged_text(tagged_text, ppl)

(0, <isanlp.annotation_repr.CSentence at 0x7f7fc4db5e50>)

In [85]:
raw_text = '[MASK] , liberals are horrible'
predictor(t)

NameError: name 't' is not defined

In [80]:
predictor.generate

AttributeError: 'MaskedTokenPredictorBert' object has no attribute 'generate'

In [104]:
import masked_token_predictor_bert
reload(masked_token_predictor_bert)
from masked_token_predictor_bert import MaskedTokenPredictorBert

from predict import analyze_tagged_text

predictor = MaskedTokenPredictorBert(model, tokenizer, max_len=250)

In [113]:
predictor._bpe_tokenizer.tokenize(tagged_text)

['_',
 '_',
 'disgusting',
 '##hy',
 '##po',
 '##cr',
 '##ites',
 '_',
 '_',
 ',',
 'liberals',
 'are',
 'mother',
 '##fu',
 '##cking',
 'horrible']

In [175]:
def convert_mask(tok_ids, mask_ids, tokenizer, duplicate=False):
    toks_tmp = [tokenizer.convert_ids_to_tokens(tok_ids[0])[1:-1]]
    mask_pos = None
    toks = []
    mask_toks = []
    has_mask = False
    for i, is_masked in enumerate(mask_ids[0][1:-1]):
        if is_masked:
            mask_toks.append(toks_tmp[0][i])
        if not has_mask:
            if is_masked:
                has_mask = True
                mask_pos = [i]
            toks.append(toks_tmp[0][i])
        else:
            if not is_masked:
                toks.extend(toks_tmp[0][i:])
                break
    toks = [toks]
    
    if duplicate:
        toks = [toks_tmp[0] + ['[SEP]'] + toks[0]]
        mask_pos[0] += len(toks_tmp[0]) + 1
    return toks, mask_pos, mask_toks

tok_ids, mask_ids = get_mask_fast('disgusting hypocrites, liberals are horrible.')
toks, mask_pos, mask_toks = convert_mask(tok_ids, mask_ids, tokenizer, duplicate=False)
print(toks, mask_pos, mask_toks)

[['disgusting', ',', 'liberals', 'are', 'horrible', '.']] [0] ['disgusting', 'h', '##yp', '##oc', '##rites']


In [159]:
predictor.generate(toks, mask_pos, n_tokens=[1, 2, 3], n_top=30)

[['disgusting', 'h', '##yp', '##oc', '##rites', ',', 'liberals', 'are', 'horrible', '.', '[SEP]', 'disgusting', ',', 'liberals', 'are', 'horrible', '.']]
[11]


([[['in', 'other', 'words'],
   'yes',
   ['in', 'any', 'event'],
   ['of', 'course'],
   ['in', 'my', 'opinion'],
   ['of', 'course', 'not'],
   'no',
   ['in', 'other', '##words'],
   ['in', 'other', 'news'],
   ['in', 'my', 'experience'],
   ['in', 'my', 'view'],
   ['in', 'any', 'case'],
   ['i', 'agree'],
   ['in', 'my', 'book'],
   ['i', 'mean'],
   ['and', 'yes'],
   ['in', 'fact'],
   ['i', 'know'],
   ['in', 'truth'],
   ['i', 'repeat'],
   ['.', 'yes'],
   ['i', 'said'],
   'well',
   'actually',
   'also',
   'indeed',
   'but',
   'sadly',
   'and',
   'so']],
 [[10.072719097137451,
   9.176839,
   8.418205261230469,
   8.19283413887024,
   8.189598242441813,
   8.045784870783487,
   8.012597,
   7.990880489349365,
   7.918407281239827,
   7.812328497568767,
   7.74386199315389,
   7.584173520406087,
   7.5116565227508545,
   7.494822343190511,
   7.481910467147827,
   7.467418909072876,
   7.275264263153076,
   6.773961305618286,
   6.773752212524414,
   6.76964259147644,


In [117]:
tagged_text = '__disgustinghypocrites__, liberals are motherfucking horrible' 

texts, scores, positions = analyze_tagged_text(tagged_text, predictor, ppl, 
                                                                         n_top=10,
                                                                         n_units=1,
                                                                         n_tokens=[1, 2, 3],
                                                                         max_multiunit=50,
                                                                         fix_multiunit=False,
                                                                         mask_token=False,
                                                                         multiunit_lookup=200)

texts

{'n_units': 1, 'n_top': 10, 'fix_multiunit': False, 'mask_token': False, 'n_tokens': [1, 2, 3], 'multiunit_lookup': 200, 'max_multiunit': 50, 'Cs': None}
[['disgusting', ',', 'liberals', 'are', 'mother', '##fu', '##cking', 'horrible']]
[0]


[['in', 'other', 'words'],
 ['in', 'my', 'opinion'],
 ['in', 'other', '##words'],
 ['in', 'my', 'view'],
 ['in', 'my', 'experience'],
 ['in', 'my', 'book'],
 'yes',
 ['in', 'other', 'news'],
 ['i', 'agree'],
 ['in', 'fact']]

In [40]:
e = embed(positions.text)
w = 3
candidates = []
for toks, score in zip(texts, scores):
    cd = cosine(e, embed(' '.join(toks)))
    candidates.append((toks, score, cd))

candidates = sorted(candidates, key = lambda s: -s[1]-s[2]*w)

In [41]:
texts

['2017',
 '2016',
 ['3', '.', 'yes'],
 ['5', '.', 'yes'],
 ['2', '.', 'yes'],
 ['4', '.', 'yes'],
 ['1', '.', 'yes'],
 ['3', '.', 'no'],
 ['5', '.', 'no'],
 ['2', '.', 'no']]

In [42]:
positions.text

'disgustinghypocrites'

In [43]:
candidates

[('2017', 7.314762, 0.0),
 ('2016', 6.9734936, 0.0),
 (['3', '.', 'yes'], 6.943620999654134, 0.0),
 (['5', '.', 'yes'], 6.915168444315593, 0.0),
 (['2', '.', 'yes'], 6.859041849772136, 0.0),
 (['4', '.', 'yes'], 6.778350989023845, 0.0),
 (['1', '.', 'yes'], 6.717874844868978, 0.0),
 (['3', '.', 'no'], 6.686502297719319, 0.0),
 (['5', '.', 'no'], 6.672283172607422, 0.0),
 (['2', '.', 'no'], 6.584537506103516, 0.0)]

In [211]:
def get_hypotheses(text, top=10, duplicate=False, mask_token=False, reorder=True, sim_coef=30):
    tokenizer = predictor._bpe_tokenizer
    tok_ids, mask_ids = get_mask_fast(text)
    toks, mask_pos, mask_toks = convert_mask(tok_ids, mask_ids, tokenizer=tokenizer, duplicate=duplicate)
    mask_text = tokenizer.convert_tokens_to_string(mask_toks)
    
    texts, scores = predictor.generate(toks, mask_pos, n_top=top,
                                                                         n_units=1,
                                                                         n_tokens=[1, 2, 3],
                                                                         max_multiunit=50,
                                                                         fix_multiunit=False,
                                                                         mask_token=mask_token,
                                                                         multiunit_lookup=200)
    texts = texts[0]
    scores = scores[0]
    
    e = embed(mask_text)
    
    candidates = [(fill_words, score, cosine(e, embed(' '.join(fill_words)))) for fill_words, score in zip(texts, scores)]
    if reorder:
        candidates = sorted(candidates, key=lambda x: x[1] + x[2] * sim_coef, reverse=True)
    return candidates

In [212]:
get_hypotheses('they are all commies who hate the usa.', duplicate=False, mask_token=True, top=50, reorder=True)

[['they', 'are', 'all', 'com', '##mies', 'who', 'hate', 'the', 'usa', '.']]
[6]


[(['want', 'war', 'on'], 9.738919576009115, 0.5378161279652228),
 (['want', 'war', 'with'], 9.051971117655436, 0.548173074628717),
 (['disagree', 'with'], 10.653357744216919, 0.48147376370278683),
 (['live', 'in'], 12.780094146728516, 0.40713618842099875),
 (['want', 'to', 'dominate'], 9.306215286254883, 0.5193634982216501),
 (['want', 'to', 'rule'], 9.457503000895182, 0.5114022576356618),
 (['live', 'out', 'of'], 9.491169134775797, 0.49341802487782876),
 (['want', 'to', 'control'], 8.995625495910645, 0.4922933683988261),
 (['have', 'come', 'to'], 10.028025309244791, 0.4577996636587268),
 (['come', 'to'], 10.062110900878906, 0.44948852617858986),
 (['live', 'outside'], 11.129491329193115, 0.4021484093942314),
 (['live', 'out', 'in'], 9.154391765594482, 0.45301393233243925),
 (['live', 'illegally', 'in'], 10.332592964172363, 0.4111384195666097),
 (['live', 'here', 'in'], 9.75853157043457, 0.42455462525992327),
 (['came', 'to'], 10.453114986419678, 0.4012123880669218),
 (['live', 'within

In [32]:
tagged_text = '__disgustinghypocrites__ , liberals are horrible' 
_analyze_tagged_text(tagged_text)

[(['*', '*', '*'], 7.921519120534261, 0.14069499121669057),
 (['.', '.', '.'], 7.655266761779785, 0.1252409178558933),
 (['=', '=', '='], 7.758999745051066, -0.003951449800087439),
 (['=', '=', '_'], 7.373194297154744, 0.05725328546334192),
 (['_', '_'], 6.9894938468933105, 0.17686531443781797),
 (['=', '='], 7.453720569610596, -0.003951459998446594),
 (['#', '#', '#'], 7.324734528859456, 0.013518912351355083),
 (['=', '_', '='], 7.1484889189402265, 0.05725326822132314),
 (['^', '^', '^'], 7.036758581797282, 0.05811795687572079),
 (['#', '#', '%'], 6.957322279612224, -0.006257355213289987)]

In [33]:
_analyze_tagged_text('disgusting hypocrites , liberals are __horrible__.')


[(['*', '*', '*'], 7.921519120534261, 0.14069499121669057),
 (['.', '.', '.'], 7.655266761779785, 0.1252409178558933),
 (['=', '=', '='], 7.758999745051066, -0.003951449800087439),
 (['=', '=', '_'], 7.373194297154744, 0.05725328546334192),
 (['_', '_'], 6.9894938468933105, 0.17686531443781797),
 (['=', '='], 7.453720569610596, -0.003951459998446594),
 (['#', '#', '#'], 7.324734528859456, 0.013518912351355083),
 (['=', '_', '='], 7.1484889189402265, 0.05725326822132314),
 (['^', '^', '^'], 7.036758581797282, 0.05811795687572079),
 (['#', '#', '%'], 6.957322279612224, -0.006257355213289987)]

In [34]:
candidates = _analyze_tagged_text('they are all commies who __hate__ the usa.')
candidates

[(['*', '*', '*'], 7.921519120534261, 0.14069499121669057),
 (['.', '.', '.'], 7.655266761779785, 0.1252409178558933),
 (['=', '=', '='], 7.758999745051066, -0.003951449800087439),
 (['=', '=', '_'], 7.373194297154744, 0.05725328546334192),
 (['_', '_'], 6.9894938468933105, 0.17686531443781797),
 (['=', '='], 7.453720569610596, -0.003951459998446594),
 (['#', '#', '#'], 7.324734528859456, 0.013518912351355083),
 (['=', '_', '='], 7.1484889189402265, 0.05725326822132314),
 (['^', '^', '^'], 7.036758581797282, 0.05811795687572079),
 (['#', '#', '%'], 6.957322279612224, -0.006257355213289987)]

In [203]:
e = embed('hate')

candidates = _analyze_tagged_text('they are all commies who __hate__ the usa.')
for c in candidates:
    print(c[0], c[1], cosine(e, embed(' '.join(c[0]))))

NameError: name '_analyze_tagged_text' is not defined

In [36]:
e = embed('disgusting hypocrites')

candidates = _analyze_tagged_text( '__disgustinghypocrites__ , liberals are horrible' )
mcd = 0
best = None
for c in candidates:
    cd = cosine(e, embed(' '.join(c[0])))
    if cd > mcd:
        mcd = cd
        best = c
    print(c[0], c[1], cd)

['*', '*', '*'] 7.921519120534261 0.015660406447927728
['.', '.', '.'] 7.655266761779785 0.03725754125051693
['=', '=', '='] 7.758999745051066 -0.05899542456714306
['=', '=', '_'] 7.373194297154744 -0.002261871014656805
['_', '_'] 6.9894938468933105 0.14047101594395606
['=', '='] 7.453720569610596 -0.058995433066475644
['#', '#', '#'] 7.324734528859456 -0.018090612088231484
['=', '_', '='] 7.1484889189402265 -0.0022618706804704712
['^', '^', '^'] 7.036758581797282 0.047306874740281016
['#', '#', '%'] 6.957322279612224 -0.04946179312835721


0.2829016397951324 (['my', 'goodness'], 6.221156477928162)


In [37]:
e = embed('imbecile')

candidates = _analyze_tagged_text( 'He is an economic __imbecile__.' )
mcd = 0
best = None
for c in candidates:
    cd = cosine(e, embed(' '.join(c[0])))
    if cd > mcd:
        mcd = cd
        best = c
    print(c[0], c[1], cd)

print(mcd, best)

['*', '*', '*'] 7.921519120534261 -0.07281956958562612
['.', '.', '.'] 7.655266761779785 -0.21645356457928608
['=', '=', '='] 7.758999745051066 -0.09866870169018566
['=', '=', '_'] 7.373194297154744 -0.13607912444422654
['_', '_'] 6.9894938468933105 -0.150988986105091
['=', '='] 7.453720569610596 -0.09866870977424569
['#', '#', '#'] 7.324734528859456 -0.07327265752233413
['=', '_', '='] 7.1484889189402265 -0.1360791249010521
['^', '^', '^'] 7.036758581797282 0.07723455771282743
['#', '#', '%'] 6.957322279612224 -0.08551658442812683
0.07723455771282743 (['^', '^', '^'], 7.036758581797282, 0.05811795687572079)
